In [16]:
import json
import numpy as np
import csv

In [17]:
google_img_str = 'http://drive.google.com/uc?export=view&id='

In [18]:
data_dict = {}
line_counter = 0
with open('highschoolers_file_ids.csv') as csv_file:
    for line in csv_file:
        if line_counter == 0:
            header_list = line.strip().split(',')
        else:
            line_data = line.strip().split(',')
            data_dict[line_data[0]] = {i: j for (i, j) in zip(header_list[1:], line_data[1:])}
        line_counter += 1
print(header_list)

['Name', 'Date', 'Size', 'URL', 'ID']


In [21]:
files = [i for i in set([i.split('_')[0] for i in data_dict.keys()]) if '.' not in i]

In [22]:
json_dict = {}
for file in files:
    img_files = [i for i in data_dict.keys() if file in i]
    orig_file = [i for i in img_files if ('original_500px' in i) & ('png' in i)][0]
    bpg_files = [i for i in img_files if ('bpg' in i) & ('png' in i)]
    for bpg_file in bpg_files:
        json_dict[bpg_file] = {
            'orig_url': google_img_str + data_dict[orig_file]['ID'],
            'img_url' : google_img_str + data_dict[bpg_file]['ID']
        }

In [25]:
with open("highschoolers_images_stimdict.json", "w") as write_file:
    json.dump(json_dict, write_file)